In [0]:
from torch.autograd import Function


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

In [0]:
def save_model(net, model_root, filename):
    """Save trained model."""
    if not os.path.exists(model_root):
        os.makedirs(model_root)
    torch.save(net.state_dict(), os.path.join(model_root, filename))
    print("save pretrained model to: {}".format(os.path.join(model_root, filename)))

In [0]:
import torch.nn as nn
from torchvision import models

class SVHNmodel(nn.Module):
    """ SVHN architecture
    """

    def __init__(self):
        super(SVHNmodel, self).__init__()
        self.restored = False

        self.feature = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(5, 5)),  # 28
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),  # 13
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(5, 5)),  # 9
            nn.BatchNorm2d(64),
            nn.Dropout2d(),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=(3, 3), stride=(2, 2)),  # 4
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(4, 4)),  # 1
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 1 * 1, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 10),
        )

        self.discriminator = nn.Sequential(
            nn.Linear(128 * 1 * 1, 1024),
            nn.BatchNorm1d(1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 2),
        )

    def forward(self, input_data, alpha = 1.0):
        input_data = input_data.expand(input_data.data.shape[0], 3, 32, 32)
        feature = self.feature(input_data)
        feature = feature.view(-1, 128 * 1 * 1)
        reverse_feature = ReverseLayerF.apply(feature, alpha)
        class_output = self.classifier(feature)
        domain_output = self.discriminator(reverse_feature)

        return class_output, domain_output


In [0]:

import torch.utils.data
import torch.nn as nn

def test(model, data_loader, device, flag):
    """Evaluate model for dataset."""
    # set eval state for Dropout and BN layers
    model.eval()

    # init loss and accuracy
    loss_ = 0.0
    acc_ = 0.0
    acc_domain_ = 0.0
    n_total = 0

    # set loss function
    criterion = nn.CrossEntropyLoss()

    # evaluate network
    for (images, labels) in data_loader:
        images = images.to(device)
        labels = labels.to(device) 
        size = len(labels)
        if flag == 'target':
            labels_domain = torch.ones(size).long().to(device)
        else:
            labels_domain = torch.zeros(size).long().to(device)

        preds, domain = model(images, alpha=0)
        loss_ += criterion(preds, labels).item()

        pred_cls = preds.data.max(1)[1]
        pred_domain = domain.data.max(1)[1]
        acc_ += pred_cls.eq(labels.data).sum().item()
        acc_domain_ += pred_domain.eq(labels_domain.data).sum().item()
        n_total += size

    loss = loss_ / n_total
    acc = acc_ / n_total
    acc_domain = acc_domain_ / n_total

    print("Avg Loss = {:.6f}, Avg Accuracy = {:.2%}, {}/{}, Avg Domain Accuracy = {:2%}".format(loss, acc, acc_, n_total, acc_domain))

    return loss, acc, acc_domain

In [0]:
def adjust_learning_rate(optimizer, p):
    lr_0 = 0.01
    alpha = 10
    beta = 0.75
    lr = lr_0 / (1 + alpha * p)**beta
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr
    return lr

def adjust_learning_rate_office(optimizer, p):
    lr_0 = 0.001
    alpha = 10
    beta = 0.75
    lr = lr_0 / (1 + alpha * p)**beta
    for param_group in optimizer.param_groups[:2]:
        param_group['lr'] = lr
    for param_group in optimizer.param_groups[2:]:
        param_group['lr'] = 10 * lr
    return lr

In [0]:
"""Train dann."""

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

def train_dann(model, params, src_data_loader, tgt_data_loader, tgt_data_loader_eval, device, logger):
    """Train dann."""
    ####################
    # 1. setup network #
    ####################

    # setup criterion and optimizer
    optimizer = optim.Adam(model.parameters(), lr=params.lr, betas=(params.beta1, 0.999))

    criterion = nn.CrossEntropyLoss()

    ####################
    # 2. train network #
    ####################
    global_step = 0
    for epoch in range(params.num_epochs):
        # set train state for Dropout and BN layers
        model.train()
        # zip source and target data pair
        len_dataloader = min(len(src_data_loader), len(tgt_data_loader))
        data_zip = enumerate(zip(src_data_loader, tgt_data_loader))
        for step, ((images_src, class_src), (images_tgt, _)) in data_zip:

            p = float(step + epoch * len_dataloader) / \
                params.num_epochs / len_dataloader
            alpha = 2. / (1. + np.exp(-10 * p)) - 1

            lr = adjust_learning_rate(optimizer, p)
            logger.add_scalar('lr', lr, global_step)

            # prepare domain label
            size_src = len(images_src)
            size_tgt = len(images_tgt)
            label_src = torch.zeros(size_src).long().to(device)  # source 0
            label_tgt = torch.ones(size_tgt).long().to(device)  # target 1

            # make images variable
            class_src = class_src.to(device)
            images_src = images_src.to(device)
            images_tgt = images_tgt.to(device)

            # zero gradients for optimizer
            optimizer.zero_grad()

            # train on source domain
            src_class_output, src_domain_output = model(input_data=images_src, alpha=alpha)
            src_loss_class = criterion(src_class_output, class_src)
            src_loss_domain = criterion(src_domain_output, label_src)

            # train on target domain
            _, tgt_domain_output = model(input_data=images_tgt, alpha=alpha)
            tgt_loss_domain = criterion(tgt_domain_output, label_tgt)


            loss = src_loss_class + src_loss_domain + tgt_loss_domain
            if params.src_only_flag:
                loss = src_loss_class

            # optimize dann
            loss.backward()
            optimizer.step()

            global_step += 1

            # print step info
            logger.add_scalar('src_loss_class', src_loss_class.item(), global_step)
            logger.add_scalar('src_loss_domain', src_loss_domain.item(), global_step)
            logger.add_scalar('tgt_loss_domain', tgt_loss_domain.item(), global_step)
            logger.add_scalar('loss', loss.item(), global_step)

            if ((step + 1) % params.log_step == 0):
                print(
                    "Epoch [{:4d}/{}] Step [{:2d}/{}]: src_loss_class={:.6f}, src_loss_domain={:.6f}, tgt_loss_domain={:.6f}, loss={:.6f}"
                    .format(epoch + 1, params.num_epochs, step + 1, len_dataloader, src_loss_class.data.item(),
                            src_loss_domain.data.item(), tgt_loss_domain.data.item(), loss.data.item()))

        # eval model
        if ((epoch + 1) % params.eval_step == 0):
            print("Train SVHN score")
            src_train_loss, src_tr_acc, src_tr_acc_domain = test(model, src_data_loader, device, flag='source')
            print("Test SVHN score")
            src_test_loss, src_acc, src_acc_domain = test(model, src_data_loader_eval, device, flag='source')
            print("Test MNIST score")
            tgt_test_loss, tgt_acc, tgt_acc_domain = test(model, tgt_data_loader_eval, device, flag='target')
            logger.add_scalar('src_test_loss', src_test_loss, global_step)
            logger.add_scalar('src_acc', src_acc, global_step)
            logger.add_scalar('src_acc_domain', src_acc_domain, global_step)
            logger.add_scalar('tgt_test_loss', tgt_test_loss, global_step)
            logger.add_scalar('tgt_acc', tgt_acc, global_step)
            logger.add_scalar('tgt_acc_domain', tgt_acc_domain, global_step)


        # save model parameters
        if ((epoch + 1) % params.save_step == 0):
            save_model(model, params.model_root, params.src_dataset + '-' + params.tgt_dataset + "-dann-{}.pt".format(epoch + 1))

    #save final model
    save_model(model, params.model_root, params.src_dataset + '-' + params.tgt_dataset + "-dann-final.pt")
    return model

In [0]:
import os
import random

import torch
import torch.backends.cudnn as cudnn
from torchvision import datasets, transforms

def init_weights(m):
    if type(m) == nn.Conv2d:
        nn.init.xavier_normal_(m.weight)
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)

def init_model(net, restore):
    """Init models with cuda and weights."""
    # init weights of model
    net.apply(init_weights)

    # check if cuda is available
    if torch.cuda.is_available():
        cudnn.benchmark = True
        net.cuda()

    return net

def init_random_seed(manual_seed):
    """Init random seed."""
    seed = None
    if manual_seed is None:
        seed = random.randint(1, 10000)
    else:
        seed = manual_seed
    print("use random seed: {}".format(seed))
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        
def get_data_loader(name, dataset_root, batch_size, train=True):
    """Get data loader by name."""
    if name == "mnist":
        return get_mnist(dataset_root, batch_size, train)
    elif name == "svhn":
        return get_svhn(dataset_root, batch_size, train)
    
def get_svhn(dataset_root, batch_size, train):
    """Get SVHN datasets loader."""
    # image pre-processing
    pre_process = transforms.Compose([transforms.Resize(32),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          mean=(0.5, 0.5, 0.5),
                                          std=(0.5, 0.5, 0.5)
                                      )])

    # datasets and data loader
    if train:
        svhn_dataset = datasets.SVHN(root=os.path.join(dataset_root),
                                   split='train',
                                   download=True,  
                                   transform=pre_process)
    else:
        svhn_dataset = datasets.SVHN(root=os.path.join(dataset_root),
                                   split='test',
                                   download=True,
                                   transform=pre_process)

    svhn_data_loader = torch.utils.data.DataLoader(
        dataset=svhn_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True)

    return svhn_data_loader

def get_mnist(dataset_root, batch_size, train):
    """Get MNIST datasets loader."""
    # image pre-processing
    pre_process = transforms.Compose([transforms.Resize(32), # different img size settings for mnist(28) and svhn(32).
                                      transforms.Grayscale(num_output_channels=3),
                                      transforms.ToTensor(),
                                      transforms.Normalize(
                                          mean=(0.5, 0.5, 0.5),
                                          std=(0.5, 0.5, 0.5)
                                      )])

    # datasets and data loader
    mnist_dataset = datasets.MNIST(root=os.path.join(dataset_root),
                                   train=train,
                                   download=True,
                                   transform=pre_process)


    mnist_data_loader = torch.utils.data.DataLoader(
        dataset=mnist_dataset,
        batch_size=batch_size,
        shuffle=True,
        drop_last=True,
        num_workers=8)

    return mnist_data_loader

In [0]:
import os
import sys
import datetime
import multiprocessing
from torch.utils import tensorboard
import torch


class Config(object):
    # params for path
    model_name = "svhn-mnist"
    model_base = '../data'
    model_root = os.path.expanduser(os.path.join('~', 'Models', 'pytorch-DANN', model_name))
    note = 'paper-structure'
    model_root = os.path.join(model_base, model_name, note + '_' + datetime.datetime.now().strftime('%m%d_%H%M%S'))
    os.makedirs(model_root)
    config = os.path.join(model_root, 'config.txt')
    finetune_flag = False
    lr_adjust_flag = 'simple'
    src_only_flag = False

    # params for datasets and data loader
    batch_size = 128

    # params for source dataset
    src_dataset = "svhn"
    src_image_root = os.path.join('../data', 'svhn')
    src_model_trained = True
    src_classifier_restore = os.path.join(model_root, src_dataset + '-source-classifier-final.pt')

    # params for target dataset
    tgt_dataset = "mnist"
    tgt_image_root = os.path.join('../data', 'mnist')
    tgt_model_trained = True
    dann_restore = os.path.join(model_root, src_dataset + '-' + tgt_dataset + '-dann-final.pt')

    # params for training dann
    gpu_id = '0'

    ## for digit
    num_epochs = 50
    log_step = 50
    save_step = 100
    eval_step = 1

    manual_seed = None
    alpha = 0

    # params for optimizing models
    lr = 0.01
    beta1 = 0.5

    def __init__(self):
        public_props = (name for name in dir(self) if not name.startswith('_'))
        with open(self.config, 'w') as f:
            for name in public_props:
                f.write(name + ': ' + str(getattr(self, name)) + '\n')

params = Config()
logger = tensorboard.SummaryWriter(params.model_root)
device = torch.device("cuda:" + params.gpu_id if torch.cuda.is_available() else "cpu")

# init random seed
init_random_seed(params.manual_seed)

# load dataset
src_data_loader = get_data_loader(params.src_dataset, params.src_image_root, params.batch_size, train=True)
src_data_loader_eval = get_data_loader(params.src_dataset, params.src_image_root, params.batch_size, train=False)
tgt_data_loader = get_data_loader(params.tgt_dataset, params.tgt_image_root, params.batch_size, train=True)
tgt_data_loader_eval = get_data_loader(params.tgt_dataset, params.tgt_image_root, params.batch_size, train=False)

0it [00:00, ?it/s]

use random seed: 5222


100%|█████████▉| 181174272/182040794 [00:15<00:00, 17099985.85it/s]
0it [00:00, ?it/s]


 97%|█████████▋| 62078976/64275384 [00:08<00:00, 17563763.94it/s]

0it [00:00, ?it/s]



  0%|          | 0/9912422 [00:00<?, ?it/s]

  0%|          | 16384/9912422 [00:00<02:28, 66455.73it/s]

  0%|          | 40960/9912422 [00:01<02:10, 75899.63it/s]

  1%|          | 98304/9912422 [00:01<01:41, 96479.41it/s]

  2%|▏         | 212992/9912422 [00:01<01:15, 127846.27it/s]

  4%|▍         | 425984/9912422 [00:01<00:54, 173096.65it/s]

  6%|▌         | 614400/9912422 [00:02<00:40, 228040.03it/s]

 10%|█         | 1024000/9912422 [00:02<00:28, 309909.95it/s]

 19%|█▉        | 1884160/9912422 [00:02<00:18, 428479.06it/s]

 35%|███▍      | 3457024/9912422 [00:02<00:10, 597227.88it/s]

 63%|██████▎   | 6234112/9912422 [00:02<00:04, 836645.80it/s]

 95%|█████████▍| 9379840/9912422 [00:03<00:00, 1166683.46it/s]

9920512it [00:03, 3198329.01it/s]                             

...

Extracting ../data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/mnist/MNIST/raw




  0%|          | 0/1648877 [00:00<?, ?it/s]

  1%|          | 16384/1648877 [00:00<00:24, 65620.11it/s]

  2%|▏         | 40960/1648877 [00:00<00:21, 74938.88it/s]

  6%|▌         | 98304/1648877 [00:01<00:16, 95246.52it/s]

 13%|█▎        | 212992/1648877 [00:01<00:11, 126215.19it/s]

 26%|██▌       | 425984/1648877 [00:01<00:07, 170883.02it/s]

 38%|███▊      | 622592/1648877 [00:01<00:04, 225837.81it/s]

 62%|██████▏   | 1024000/1648877 [00:02<00:02, 306478.54it/s]

1654784it [00:02, 816172.58it/s]                             

0it [00:00, ?it/s]

Extracting ../data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/mnist/MNIST/raw




  0%|          | 0/4542 [00:00<?, ?it/s]

8192it [00:00, 18705.81it/s]            

Extracting ../data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/mnist/MNIST/raw
Processing...
Done!


In [0]:
# load dann model
dann = init_model(net=SVHNmodel(), restore=None)

# train dann model
print("Training dann model")
dann = train_dann(dann, params, src_data_loader, tgt_data_loader, tgt_data_loader_eval, device, logger)


64282624it [00:22, 17563763.94it/s]                              

Training dann model
Epoch [   1/50] Step [50/468]: src_loss_class=1.794217, src_loss_domain=0.505048, tgt_loss_domain=0.509467, loss=2.808732
Epoch [   1/50] Step [100/468]: src_loss_class=1.230935, src_loss_domain=0.280223, tgt_loss_domain=0.190053, loss=1.701211
Epoch [   1/50] Step [150/468]: src_loss_class=0.943820, src_loss_domain=0.066955, tgt_loss_domain=0.262523, loss=1.273298
Epoch [   1/50] Step [200/468]: src_loss_class=0.866094, src_loss_domain=0.060876, tgt_loss_domain=0.130522, loss=1.057492
Epoch [   1/50] Step [250/468]: src_loss_class=0.723378, src_loss_domain=0.671953, tgt_loss_domain=0.039317, loss=1.434648
Epoch [   1/50] Step [300/468]: src_loss_class=0.628017, src_loss_domain=0.167799, tgt_loss_domain=0.225468, loss=1.021284
Epoch [   1/50] Step [350/468]: src_loss_class=0.847219, src_loss_domain=0.113404, tgt_loss_domain=0.182628, loss=1.143251
Epoch [   1/50] Step [400/468]: src_loss_class=0.731506, src_loss_domain=0.514285, tgt_loss_domain=0.614318, loss=1.8601